In [3]:
from z3 import *

from itertools import combinations
from z3 import *
from utils import *
import math

In [4]:
def at_least_one_np(bool_vars):
    return Or(bool_vars)

def at_most_one_np(bool_vars, name = ""):
    return And([Not(And(pair[0], pair[1])) for pair in combinations(bool_vars, 2)])

def exactly_one_np(bool_vars, name = ""):
    return And(at_least_one_np(bool_vars), at_most_one_np(bool_vars, name))

In [5]:
def at_least_one_seq(bool_vars):
    return at_least_one_np(bool_vars)

def at_most_one_seq(bool_vars, name):
    constraints = []
    n = len(bool_vars)
    s = [Bool(f"s_{name}_{i}") for i in range(n - 1)]
    constraints.append(Or(Not(bool_vars[0]), s[0]))
    constraints.append(Or(Not(bool_vars[n-1]), Not(s[n-2])))
    for i in range(1, n - 1):
        constraints.append(Or(Not(bool_vars[i]), s[i]))
        constraints.append(Or(Not(bool_vars[i]), Not(s[i-1])))
        constraints.append(Or(Not(s[i-1]), s[i]))
    return And(constraints)

def exactly_one_seq(bool_vars, name):
    return And(at_least_one_seq(bool_vars), at_most_one_seq(bool_vars, name))

In [71]:
def at_least_k_seq(bool_vars, k, name):
    return at_most_k_seq([Not(var) for var in bool_vars], len(bool_vars)-k, name)

def at_most_k_seq(bool_vars, k, name):
    constraints = []
    n = len(bool_vars)
    s = [[Bool(f"s_{name}_{i}_{j}") for j in range(k)] for i in range(n - 1)]
    constraints.append(Or(Not(bool_vars[0]), s[0][0]))
    constraints += [Not(s[0][j]) for j in range(1, k)]
    for i in range(1, n-1):
        constraints.append(Or(Not(bool_vars[i]), s[i][0]))
        constraints.append(Or(Not(s[i-1][0]), s[i][0]))
        constraints.append(Or(Not(bool_vars[i]), Not(s[i-1][k-1])))
        for j in range(1, k):
            constraints.append(Or(Not(bool_vars[i]), Not(s[i-1][j-1]), s[i][j]))
            constraints.append(Or(Not(s[i-1][j]), s[i][j]))
    constraints.append(Or(Not(bool_vars[n-1]), Not(s[n-2][k-1])))   
    return And(constraints)

def exactly_k_seq(bool_vars, k, name):
    return And(at_most_k_seq(bool_vars, k, name+'1'), at_least_k_seq(bool_vars, k, name))

In [74]:
'''n packs
   m couriers
   D matix int(n+1,m+1)
   sz array of item's sizes
   cpt array of couriers' capacities
'''
def mcp(n, m, D, sz, cpt):
   
   #pred[i][j] = true if the j-th item is the predecessor of the i-th item
   pred = [[Bool(f"pred({i})_{j}")for j in range(n+1)]for i in range(n+1)]
   #print(pred)

   #create Solver
   solv= Solver()

   col_n = []
   for j in range(n):
      col_n += [pred[j][n]]
   solv.add(exactly_k_seq(pred[n],m, f"PR_{n}")) #pred row
   solv.add(exactly_k_seq(col_n,m, f"PC_{n}")) #pred col
   print(solv.check())


   #i=0 to i=n-1 constraint At most one
   #j=0 to j=n-1 constraint At most one
   for i in range(n):
      col_i = []
      for j in range(n+1):
         col_i += [pred[j][i]]

      solv.add(exactly_one_seq(col_i, f"PC_{i}"))
      print(pred[i]) 
      solv.add(exactly_one_seq(pred[i], f"PR_{i}"))

   



   #create matrix v
   v=[[Bool(f"vehicle({i})_{j}")for j in range(m)]for i in range(n)]
   #print(v)

   #each pack have to be carried by exactly one courrier
   for i in range(n):
      solv.add(exactly_one_seq(v[i], f"v_{i}"))
   
   print(solv.check())
   mod = solv.model()
   print(mod)
   print(mod.evaluate(pred[0][0]))
   return [(i, j) for i in range(n) for j in range(n) if mod.evaluate(pred[i][j])]

print(mcp(10,3,0,0,0))



sat
[pred(0)_0, pred(0)_1, pred(0)_2, pred(0)_3, pred(0)_4, pred(0)_5, pred(0)_6, pred(0)_7, pred(0)_8, pred(0)_9, pred(0)_10]
[pred(1)_0, pred(1)_1, pred(1)_2, pred(1)_3, pred(1)_4, pred(1)_5, pred(1)_6, pred(1)_7, pred(1)_8, pred(1)_9, pred(1)_10]
[pred(2)_0, pred(2)_1, pred(2)_2, pred(2)_3, pred(2)_4, pred(2)_5, pred(2)_6, pred(2)_7, pred(2)_8, pred(2)_9, pred(2)_10]
[pred(3)_0, pred(3)_1, pred(3)_2, pred(3)_3, pred(3)_4, pred(3)_5, pred(3)_6, pred(3)_7, pred(3)_8, pred(3)_9, pred(3)_10]
[pred(4)_0, pred(4)_1, pred(4)_2, pred(4)_3, pred(4)_4, pred(4)_5, pred(4)_6, pred(4)_7, pred(4)_8, pred(4)_9, pred(4)_10]
[pred(5)_0, pred(5)_1, pred(5)_2, pred(5)_3, pred(5)_4, pred(5)_5, pred(5)_6, pred(5)_7, pred(5)_8, pred(5)_9, pred(5)_10]
[pred(6)_0, pred(6)_1, pred(6)_2, pred(6)_3, pred(6)_4, pred(6)_5, pred(6)_6, pred(6)_7, pred(6)_8, pred(6)_9, pred(6)_10]
[pred(7)_0, pred(7)_1, pred(7)_2, pred(7)_3, pred(7)_4, pred(7)_5, pred(7)_6, pred(7)_7, pred(7)_8, pred(7)_9, pred(7)_10]
[pred(8)_0, 